In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as soup

# Data Preparation

## 0 - 30 tourist sites

In [6]:
html_main = requests.get("https://www.tripadvisor.com/Attractions-g32655-Activities-a_allAttractions.true-Los_Angeles_California.html")
bsobj_main = soup(html_main.content, "lxml")

In [7]:
rank = []
tourist_site_name = []
tourist_site_link = []
for s in bsobj_main.find_all("section", class_="_2TabEHya _3YhIe-Un"):
    rank.append(s.find("div", class_="_1gpq3zsA _1zP41Z7X").text.split(". ")[0])
    tourist_site_name.append(s.find("div", class_="_1gpq3zsA _1zP41Z7X").text.split(". ")[1])
    tourist_site_link.append(s.find("div", class_="_3W_31Rvp _1nUIPWja _1l7Rsl_O _3ksqqIVm _2b3s5IMB").a["href"])

## 31 - 150 tourist sites

In [8]:
top_150 = list(range(30, 150, 30))

In [9]:
for num in top_150:
    html_main = requests.get("https://www.tripadvisor.com/Attractions-g32655-Activities-oa" + str(num) + "-Los_Angeles_California.html")
    bsobj_main = soup(html_main.content, "lxml")
    
    for s in bsobj_main.find_all("section", class_="_2TabEHya _3YhIe-Un"):
        rank.append(s.find("div", class_="_1gpq3zsA _1zP41Z7X").text.split(". ")[0])
        tourist_site_name.append(s.find("div", class_="_1gpq3zsA _1zP41Z7X").text.split(". ")[1])
        tourist_site_link.append(s.find("div", class_="_3W_31Rvp _1nUIPWja _1l7Rsl_O _3ksqqIVm _2b3s5IMB").a["href"])

In [10]:
tourist_site_link_update = []
for site in tourist_site_link:
    tourist_site_link_update.append("https://www.tripadvisor.com/" + site)

In [11]:
data = {"Rank": rank, "Tourist Site Name": tourist_site_name, "Site Link": tourist_site_link_update}
df = pd.DataFrame.from_dict(data)
df

,Rank,Tourist Site Name,Site Link
0,1,The Getty Center,https://www.tripadvisor.com//Attraction_Review...
1,2,Griffith Observatory,https://www.tripadvisor.com//Attraction_Review...
2,3,Universal Studios Hollywood,https://www.tripadvisor.com//Attraction_Review...
3,4,Petersen Automotive Museum,https://www.tripadvisor.com//Attraction_Review...
4,5,The Wizarding World of Harry Potter,https://www.tripadvisor.com//Attraction_Review...
...,...,...,...
145,146,S.S,https://www.tripadvisor.com//Attraction_Review...
146,147,Kenneth Hahn State Recreation Area,https://www.tripadvisor.com//Attraction_Review...
147,148,Geffen Playhouse,https://www.tripadvisor.com//Attraction_Review...
148,149,Los Angeles Police Museum,https://www.tripadvisor.com//Attraction_Review...


# Location After Input

In [12]:
tourist_site_name

['The Getty Center',
 'Griffith Observatory',
 'Universal Studios Hollywood',
 'Petersen Automotive Museum',
 'The Wizarding World of Harry Potter',
 'Battleship USS Iowa Museum',
 'The Broad',
 'Staples Center',
 'Griffith Park',
 'The Grove',
 'La Brea Tar Pits and Museum',
 'Walt Disney Concert Hall',
 'Natural History Museum of Los Angeles County',
 'Runyon Canyon Park',
 'Venice Canals Walkway',
 'The Nethercutt Collection',
 'Dodger Stadium',
 'Union Station',
 'Hollywood Sign',
 'Bradbury Building',
 'Lake Hollywood Park',
 'Universal CityWalk Hollywood',
 'Los Angeles County Museum of Art',
 'Madame Tussauds Hollywood',
 'Abbot Kinney Boulevard',
 'Angels Flight Railway',
 'University of California, Los Angeles (UCLA)',
 'Hollywood Forever Cemetery',
 'Citadel Outlets',
 'Pantages Theatre',
 'The Hollywood Museum',
 'OUE Skyspace LA',
 'Los Angeles Central Library',
 'Olvera Street',
 'Pierce Brothers Westwood Village Memorial Park',
 'Dolby Theatre',
 'The Greek Theatre',
 'Ci

*** may update want_to_go_name to make it consider capital letter/ error message/ give example

In [14]:
def visit_tourist(df, want_to_go_name):
    each_name = want_to_go_name.split(".")
    
    for name in each_name:
        if name == each_name[0]:
            visit_html = df[["Tourist Site Name", "Site Link"]][df["Tourist Site Name"] == name]
        else:
            visit_html = visit_html.append(df[["Tourist Site Name", "Site Link"]][df["Tourist Site Name"] == name])
    
    return visit_html

In [19]:
want_to_go_name = str(input("Which tourist sites do you want to go? (*please input the names of the tourist sites separated by a period)"))
visit_html = visit_tourist(df, want_to_go_name)

Which tourist sites do you want to go? (*please input the names of the tourist sites separated by a period)Chinatown.Geffen Playhouse.


In [20]:
def find_location(visit_html):
    
    location = []
    
    for link in visit_html["Site Link"]:
        html = requests.get(link)
        bsobj = soup(html.content, "lxml")
    
        for loc in bsobj.find_all("script", type = "application/ld+json"):
            if "streetAddress" in loc.string:
                find_part_html = loc.string.split("{")
                for street in find_part_html:
                    if "streetAddress" in street:
                        for value in street.split(","):
                            if "streetAddress" in value:
                                street_name = value.split(":")[1][1:][:-1] + ", "
                                   
                            if "addressLocality" in value:
                                city_name = value.split(":")[1][1:][:-1] + ", "
                                 
                            if "postalCode" in value:
                                zipcode = "CA " + value.split(":")[1][1:][:-1]
                            
        location_info = street_name + city_name + zipcode
        location.append(location_info)
    
    return location

In [21]:
location = find_location(visit_html)

In [22]:
location

['N Broadway at North Hill St Downtown, Los Angeles, CA 90012',
 '10886 Le Conte Ave, Los Angeles, CA 90024-3098']